# MovieTinder

file to do the cleaning, while Jupiter notebook has better visualization for tables.

In [37]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import ast

df = pd.read_csv('data/movies_metadata_org.csv')
print("Original shape: ", df.shape)


# drop the columns which have a missing value percentage of above 55 
toBeDropped = df.columns[df.isnull().mean() > 0.55] 
df = df.drop(columns= toBeDropped)
print("Shape after dropping NaN columns: ", df.shape, " Columns deleted: ", len(toBeDropped))

# from this we can see that most rows are complete (without missing values) while only 10 rows have less than 21 finite values in their row with 21
# being the total amount of colums, therefore we do not drop any rows found by this. 
# print(len(df.dropna(thresh=20)))

title = df['title']
df = df.drop(columns = ['original_title', 'title', 'status', 'video', 'poster_path', 'production_countries','spoken_languages'])
df.insert(0, 'Title', title) #move title to the front
print("Shape after dropping unwanted columns: ", df.shape)

df = df[df['production_companies'].notna()]

#  rows which have a boolean type value for their production companies, yes i did this manualy and yes it could have been automated
df = df.drop(index = 29503)
df = df.drop(index = 19730)
df = df.drop(index = 35587)


# clean the genres and production companies columns

for index, row in df.iterrows():
    res = ast.literal_eval(row['genres']) 
    res2 = ast.literal_eval(row['production_companies'])
    genres = []
    companies = []
    
    
    for genre in res:
        genres.append(genre['name'])
    
    for comp in res2:
        companies.append(comp['name'])
    
    if len(companies) == 0:
        df.at[index,'production_companies'] = np.nan
    else: 
        df.at[index,'production_companies'] = companies
    
    if len(genres) == 0:
        df.at[index,'genres'] = np.nan
    else: 
        df.at[index, 'genres'] = genres

# drop na in genre and na in imdb_id
df = df[df['genres'].notna()]
df = df[df['imdb_id'].notna()]
df['production_companies'] = df['production_companies'].fillna(0)


Original shape:  (45466, 24)
Shape after dropping NaN columns:  (45466, 21)  Columns deleted:  3
Shape after dropping unwanted columns:  (45466, 15)


0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
5        [Regency Enterprises, Forward Pass, Warner Bros.]
6        [Paramount Pictures, Scott Rudin Productions, ...
7                                   [Walt Disney Pictures]
8        [Universal Pictures, Imperial Entertainment, S...
9                        [United Artists, Eon Productions]
10          [Columbia Pictures, Castle Rock Entertainment]
11       [Columbia Pictures, Castle Rock Entertainment,...
12       [Universal Pictures, Amblin Entertainment, Amb...
13       [Hollywood Pictures, Cinergi Pictures Entertai...
14       [Le Studio Canal+, Laurence Mark Productions, ...
15       [Universal Pictures, Légende Entreprises, Syal...
16       [Columbia Pictures Corporation, Mirage Enterpr.

In [38]:
df.isnull().mean()

Title                   0.000000
adult                   0.000000
budget                  0.000000
genres                  0.000000
id                      0.000000
imdb_id                 0.000000
original_language       0.000233
overview                0.016161
popularity              0.000000
production_companies    0.227224
release_date            0.000581
revenue                 0.000000
runtime                 0.004116
vote_average            0.000000
vote_count              0.000000
dtype: float64

In [30]:
print(df.vote_count.mean())
print(df.vote_count.max())

# add weightedRating column in the dataset to take into account the user votes and the rating, just a simple multiplication 
df['weightedRating'] = df['vote_count'] * df['vote_average']
print("length = ", len(df[df.weightedRating > 1000]))

# plt.xlim([1000,6000])
# plt.hist(df.weightedRating, bins = 100)
# plt.show()

148.8337744516443
14075.0
length =  4538


In [18]:
df.to_csv('data/movieData.csv', sep=',', encoding='utf-8', index = False)

In [16]:
# check if everything went fine
df = pd.read_csv('data/movieData.csv',error_bad_lines=False)
df.head()

,Title,adult,budget,genres,id,imdb_id,original_language,overview,popularity,production_companies,release_date,revenue,runtime,vote_average,vote_count,weightedRating
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
